In [ ]:
import os
import numpy as np

from netCDF4 import Dataset, date2num
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cf
import warnings
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from datetime import datetime, timedelta
from collections import defaultdict
from shapely import vectorized

warnings.filterwarnings(action='ignore')

### 1. 각 기후 모델별 mean, std, max 값 계산

In [ ]:

GCMs_list = ['GFDL-ESM4', 'IPSL-CM6A-LR', 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'UKESM1-0-LL']
data_types = ["historical", "picontrol"]

shapefile_path = "./shape_file_10m_cultural/ne_10m_admin_0_countries.shp"
gdf_borders = gpd.read_file(shapefile_path)
korea = gdf_borders[gdf_borders['SOVEREIGNT'].isin(['South Korea', 'North Korea'])].geometry.unary_union

lon = np.load("../data/ISIMIP_ko_lon_2km.npy")  # (601,)
lat = np.load("../data/ISIMIP_ko_lat_2km.npy")  # (601,)

lon_grid, lat_grid = np.meshgrid(lon, lat)
mask = vectorized.contains(korea, lon_grid, lat_grid)

for data_type_ in data_types:
    for GCM in GCMs_list:
        # 📌 데이터가 저장된 경로
        data_dir = f"../result/HR/{data_type_}/{GCM}/origin_fwi"
        result_path = f"../result/{data_type_}/{GCM}"
        # 📌 날짜별 데이터를 저장할 딕셔너리
        daily_data = defaultdict(list)

        # 📌 모든 파일을 가져와 월/일별 그룹화
        for file_name in sorted(os.listdir(data_dir)):
            if file_name.endswith(".npy") and file_name.startswith("20"):  # 2000~2014년 데이터만 필터링
                mmdd = file_name[4:8]  # 'MMDD' 추출 (ex: 0101, 0215, ...)
                if mmdd == "0229":
                    continue
                file_path = os.path.join(data_dir, file_name)

                data = np.load(file_path)  # (601, 601) 형태

                data = np.where(np.flipud(mask), data, np.nan)
                daily_data[mmdd].append(data)

        # 📌 결과를 저장할 배열 (365일 또는 366일)
        num_days = len(daily_data)  # 365 또는 366
        sample_shape = list(daily_data.values())[0][0].shape  # (601, 601)
        daily_mean = np.zeros((num_days, *sample_shape), dtype=np.float32)
        daily_std = np.zeros((num_days, *sample_shape), dtype=np.float32)
        daily_max = np.zeros((num_days, *sample_shape), dtype=np.float32)

        # 📌 각 월/일별 평균 & 표준편차 계산
        for i, (mmdd, data_list) in enumerate(sorted(daily_data.items())):
            data_stack = np.stack(data_list)  # (15, 601, 601)
            daily_mean[i] = np.mean(data_stack, axis=0)  # 평균 계산
            daily_std[i] = np.std(data_stack, axis=0)  # 표준편차 계산
            daily_max[i] = np.max(data_stack, axis=0)  # max 계산
            print(f"✅ {mmdd} 완료: {len(data_list)}년 데이터 처리됨")

        # 📌 최종 결과 저장
        np.save(os.path.join(result_path, "daily_mean_ko.npy"), daily_mean)
        np.save(os.path.join(result_path, "daily_std_ko.npy"), daily_std)
        np.save(os.path.join(result_path, "daily_max_ko.npy"), daily_max)

print("🎯 모든 월/일별 mean, std, max 계산 완료 및 저장됨!")

### 2. 기후 모델 취합

In [ ]:

# GCM 리스트
GCMs_list = ['GFDL-ESM4', 'IPSL-CM6A-LR', 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'UKESM1-0-LL']

# 결과를 누적할 리스트
data_list = []

# 각 GCM별로 파일 로드 (파일 경로는 예시로 구성됨)
for gcm in GCMs_list:
    file_path = f"/lustre/home/ebcho/workspace/pyfwi/SR_ISIMIP/result/historical/{gcm}/daily_max_ko.npy"
    try:
        arr = np.load(file_path)  # (365, 601, 601)
        data_list.append(arr)
    except Exception as e:
        print(f"❌ {gcm} 파일 로드 실패:", e)

# 평균 계산
if data_list:
    stacked = np.stack(data_list, axis=0)  # shape: (5, 365, 601, 601)
    mean_result = np.mean(stacked, axis=0)  # shape: (365, 601, 601)
else:
    mean_result = None
    print("📂 유효한 데이터 없음")

np.save("/lustre/home/ebcho/workspace/pyfwi/SR_ISIMIP/result/historical/all_gcm_fwi.npy", mean_result)


### 3. nc 파일로 저장

In [ ]:

# 🔹 데이터 경로 설정
fwi_path = "/lustre/home/ebcho/workspace/pyfwi/SR_ISIMIP/result/historical/all_gcm_fwi.npy"
output_nc_path = "/lustre/home/ebcho/workspace/pyfwi/SR_ISIMIP/result/historical/all_gcm_fwi.nc"


# 🔹 첫 번째 파일 로드하여 shape 확인
all_fwi = np.load(fwi_path)
lat_dim, lon_dim = all_fwi[0].shape[0], all_fwi[0].shape[1]


# 🔹 위도, 경도 데이터 로드
lat = np.load("./data/ISIMIP_ko_lat_2km.npy")
lon = np.load("./data/ISIMIP_ko_lon_2km.npy")

mean_std_times = [datetime(2000, 1, 1) + timedelta(days=i) for i in range(365)]
day_of_year = np.arange(1, 366)  # 1~365일
# print(len(mean_std_times), len(day_of_year))

# 📌 NetCDF 파일 생성 (`xarray.Dataset`)
ds = xr.Dataset(
    {
        "FWI_all": (["time_mean", "lat", "lon"], all_fwi),  # FWI (2000~2014)
        "day_of_year": (["time_mean"], day_of_year),  # 추가된 day_of_year 변수
    },
    coords={
        "time_mean": mean_std_times,  # 1년 365일 (2000년 기준)
        "lat": lat,
        "lon": lon,
    }
)

# 📌 NetCDF 파일 저장
ds.to_netcdf(output_nc_path)
print(f"✅ NetCDF 파일 저장 완료: {output_nc_path}")
